In [1]:
from fragmenstein import mpro_data

In [2]:
import pandas as pd
submissions:pd.DataFrame = mpro_data.fetch_postera()

/Users/matteo/Coding/Fragmenstein/fragmenstein/mpro/data/__init__.py:98: DtypeWarning: Columns (3,6,7,9,10,11,12,13,14,15,18,19,27,28,29,30,31,32,33,34,35,36,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,66,67,69,70,71,72,73,74,75,76,84,85,86,87,89,90,91,92,93,94,95,96,104,105,106,107,109,110,111,112,113,114,115,116,124,125,126,127,129,130,131,132,133,134,135,136,144,145,146,147,149,150,151,152,153,154,155,156,164,165,166,167,169,170,171,172,173,174,175,176,184,185,186,187,189,190,191,192,193,194,195,196,204,205,206,207,209,210,211,212,213,214,215,216,224,225,226,227,229,230,231,232,233,234,235,236,244,245,246,247,249,253,254,255,256,257,258,259,275,276) have mixed types. Specify dtype option on import or set low_memory=False.
  postera = pd.read_csv(io.StringIO(s.decode('utf-8')))


In [44]:
submissions

,SMILES,CID,CID (canonical),CID (old format),Postera Link,InChIKey,creator,rationale,Submission Notes,fragments,Structure ID,Fragalysis Link,Enamine - REAL Space,Enamine - Extended REAL Space,Enamine - SCR,Enamine - BB,Mcule,Mcule Ultimate,In Molport or Mcule,In eMolecules,MW,cLogP,TPSA,Rotatable Bonds,Fraction sp3,HBA,HBD,Covalent Fragment,covalent_warhead,Acrylamide,Acrylamide Adduct,Chloroacetamide,Chloroacetamide Adduct,Vinylsulfonamide,Vinylsulfonamide Adduct,Nitrile,Nitrile Adduct,series,Order Date,Maker,...,f_max_inhibition_reading_y.1.1.1.1,f_min_inhibition_reading_y.1.1.1.1,f_hill_slope_y.1.1.1.1,f_R2_y.1.1.1.1,f_concentration_uM_y.1.1.1.1,f_inhibition_list_y.1.1.1.1,structure_ID_y.1.1.1.1,structure_LINK_y.1.1.1.1,creator.2,rationale.2,fragments.2,covalent_warhead.2,r_inhibition_at_20_uM,r_inhibition_at_50_uM,r_avg_IC50,r_curve_IC50,r_max_inhibition_reading,r_min_inhibition_reading,r_hill_slope,r_R2,r_concentration_uM,r_inhibition_list,f_inhibition_at_20_uM,f_inhibition_at_50_uM,f_avg_IC50,f_avg_pIC50,f_curve_IC50,f_max_inhibition_reading,f_min_inhibition_reading,f_hill_slope,f_R2,f_concentration_uM,f_inhibition_list,relative_solubility_at_20_uM,relative_solubility_at_100_uM,trypsin_IC50,NMR_std_ratio,structure_ID,structure_LINK,category
0,N#Cc1ccccc1NC(=O)Cc1c[nH]c2ncccc12,ANT-DIA-3c79be55-1,ANT-DIA-3c79be55-1,ANT-DIA-3c7-1,https://covid.postera.ai/covid/submissions/3c7...,JNRKVADDLOBVCB-UHFFFAOYSA-N,"Anthony Aimon, Diamond Light Source Ltd",Nitrile of X0305 seems to be key and superimpo...,Nitrile and isosters should be tested,"x0072,x0305,x0387,x0434,x0678,x0830,x1249",NaN,NaN,False,False,False,False,MCULE-2614803204,False,False,True,276.299,2.61578,81.57,3.0,0.06,3.0,2.0,True,True,False,False,False,False,False,False,True,False,3-aminopyridine-like,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nitrile
1,N#Cc1ccccc1NC(=O)Cc1cccnc1,ANT-DIA-3c79be55-2,ANT-DIA-3c79be55-2,ANT-DIA-3c7-2,https://covid.postera.ai/covid/submissions/3c7...,LVRULDIVMIRLIV-UHFFFAOYSA-N,"Anthony Aimon, Diamond Light Source Ltd",Nitrile of X0305 seems to be key and superimpo...,Nitrile and isosters should be tested,"x0072,x0305,x0387,x0434,x0678,x0830,x1249",NaN,NaN,Z1172500933,s_11____51736____7459854,False,False,False,LVRULDIVMIRLIV-UHFFFAOYSA-N,False,False,237.262,2.13448,65.78,3.0,0.07,3.0,1.0,True,True,False,False,False,False,False,False,True,False,3-aminopyridine-like,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nitrile
2,CCNc1ccc(C#N)c(NC(=O)Cc2c[nH]c3ncccc23)c1,ANT-DIA-3c79be55-3,ANT-DIA-3c79be55-3,ANT-DIA-3c7-3,https://covid.postera.ai/covid/submissions/3c7...,LMQZSJWRDIGOJW-UHFFFAOYSA-N,"Anthony Aimon, Diamond Light Source Ltd",Nitrile of X0305 seems to be key and superimpo...,Nitrile and isosters should be tested,"x0072,x0305,x0387,x0434,x0678,x0830,x1249",NaN,NaN,False,False,False,False,False,False,False,False,319.368,3.04758,93.60,5.0,0.17,4.0,3.0,True,True,False,False,False,False,False,False,True,False,3-aminopyridine-like,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nitrile
3,CS(=O)(=O)Cc1ccc(C(=O)Nc2cccnc2)o1,ANT-DIA-3c79be55-4,ANT-DIA-3c79be55-4,ANT-DIA-3c7-4,https://covid.postera.ai/covid/submissions/3c7...,ZTEMFLQYLWQFJF-UHFFFAOYSA-N,"Anthony Aimon, Diamond Light Source Ltd",Nitrile of X0305 seems to be key and superimpo...,Nitrile and isosters should be tested,"x0072,x0305,x0387,x0434,x0678,x0830,x1249",NaN,NaN,Z995832158,False,False,False,False,False,False,False,280.305,1.47150,89.27,4.0,0.17,5.0,1.0,True,False,False,False,False,False,False,False,False,False,NaN,2020-03-31,enamine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
isoster replacement

In [14]:
import pandas as pd
results = pd.read_csv('../Fragmenstein/documentation/notes/mpro_manual.csv')
results

,cid,crystal,inspiration_hits,rmsd_via_combine,n_inspirations,rmsd_via_place,n_inspiration_str,category,SMILES,minimized2hits_rmsd,reference2hits_rmsd,reference2minimized_rmsd,ddG,num_heavy_atoms,LE
0,DAR-DIA-23aa0b97-6,x10178,"x0305,x0434",0.850404,2,6.704262,two,Nitrile,N#Cc1cncc(NC(=O)Cc2cccnc2)c1,0.396117,5.934000,7.084449,-2.311692,18,-0.128427
1,DAR-DIA-23aa0b97-13,x2964,"x0104,x0434,x0995",0.485471,3,0.478219,three,non-covalent,Nc1cncc(NC(=O)Nc2cccc(Cl)c2)c1,0.660813,0.478219,0.748774,-2.365326,18,-0.131407
2,DAR-DIA-23aa0b97-14,x10996,"x0104,x0434,x0995",1.261876,3,0.312711,three,non-covalent,O=C(Nc1cccnc1)Nc1cccc(Cl)c1,0.648168,1.235790,1.702625,-1.839026,17,-0.108178
3,DAR-DIA-23aa0b97-17,x2569,"x0305,x0434",0.367349,2,0.549061,two,Nitrile,N#Cc1cncc(CC(=O)Nc2cccnc2)c1,0.621199,0.549061,0.508123,-1.360511,18,-0.075584
4,DAR-DIA-23aa0b97-20,x2563,"x0434,x1093",1.933083,2,1.483181,two,non-covalent,O=C(Cc1cncc2ccccc12)Nc1ccccc1,1.465375,1.672828,1.603740,-0.660797,20,-0.033040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,EDJ-MED-6af13d92-2,x11276,x0678,NaN,1,NaN,one,non-covalent,COc1ccccc1OCCNC(=O)c1cc(=O)[nH]c2cccc(F)c12,NaN,NaN,NaN,NaN,26,NaN
96,EDJ-MED-6af13d92-3,x11294,x0678,8.202195,1,7.817587,one,non-covalent,COc1ccccc1OCCNC(=O)c1cc(=O)[nH]c2cccc(OC)c12,1.335216,7.577411,7.790441,0.000000,27,0.000000
97,MIC-UNK-66895286-1,x11532,x0678,NaN,1,NaN,one,non-covalent,O=C(Cc1cccc(Cl)c1)Nc1cn[nH]c1,NaN,NaN,NaN,NaN,16,NaN
98,MIC-UNK-66895286-3,x11540,x0678,NaN,1,NaN,one,non-covalent,Cc1[nH]ncc1CC(=O)Nc1cccc(Cl)c1,NaN,NaN,NaN,NaN,17,NaN


In [21]:
pd.pivot_table(results, 'reference2minimized_rmsd', 'category')

,reference2minimized_rmsd
category,
Nitrile,3.148228
non-covalent,4.547901


In [27]:
results['correct'] = results.reference2minimized_rmsd > 1

In [23]:
pd.pivot_table(results, 'correct', 'category')

,correct
category,
Chloroacetamide,0.000000
Nitrile,0.714286
non-covalent,0.852273


In [50]:
migrate_col = lambda col: results['cid'].map(submissions[['CID', col]].set_index('CID')[col].fillna('NA').to_dict())
results['rationale'] = migrate_col('rationale') + ' ' + migrate_col('Submission Notes')

In [69]:
from IPython.display import display

results['docked'] = results.rationale.str.contains(r'[Dd]ock') | results.rationale.str.contains(r'[Ll]ibrary')  | results.rationale.str.contains(r'[sS]can')
results['eye'] = results.rationale.str.contains('eye') | results.rationale.str.contains('superimpo')
display()
df = pd.pivot_table(results, 'correct', 'eye')
df['counts'] = results['eye'].value_counts()
display(df)
df = pd.pivot_table(results, 'correct', 'docked')
df['counts'] = results['docked'].value_counts()
display(df)

,correct,counts
eye,,
False,0.797468,79
True,0.809524,21


,correct,counts
docked,,
False,0.806452,62
True,0.789474,38


In [84]:
from collections import Counter
import re

success = {}
counts = {}
for word, count in Counter( re.findall(r'(\w+)', ' '.join(results.rationale.values)) ).most_common():
    results['_'] = results.rationale.str.contains(word)
    df = pd.pivot_table(results, 'correct', '_')
    df['counts'] = results['_'].value_counts()
    if df.loc[True, 'counts'] > 5:
        success[word] = df.loc[True, 'correct']
        counts[word] = df.loc[True, 'counts']

In [89]:
pd.DataFrame(dict(success=success, counts=counts)).sort_values('success').tail(20)

,success,counts
acetamide,1.0,6
pi,1.0,11
23,1.0,8
x_0689,1.0,9
Simon,1.0,7
screen,1.0,8
68,1.0,13
39,1.0,13
side,1.0,7
have,1.0,6


In [90]:
results.loc[results.crystal == 'x10728']

,cid,crystal,inspiration_hits,rmsd_via_combine,n_inspirations,rmsd_via_place,n_inspiration_str,category,SMILES,minimized2hits_rmsd,reference2hits_rmsd,reference2minimized_rmsd,ddG,num_heavy_atoms,LE,correct,rationale,docked,eye,_
50,WAR-XCH-72a8c209-5,x10728,x1493,1.861304,1,4.988635,one,Chloroacetamide,O=C(CCl)N1CCCC(N(CC2CCCCC2)C2CCC(O)CC2)C1,NaN,NaN,NaN,-1.341399,25,-0.053656,False,1. Data combo of top 40 scoring docking compou...,True,False,False
